## Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from typing import Tuple
import sys
from pathlib import Path
from datetime import datetime
import os
import pyro

# Add parent directory to path to import Models
# This works for notebooks in the Experiments folder
project_root = Path.cwd().parent if Path.cwd().name == 'Experiments' else Path.cwd()
sys.path.insert(0, str(project_root))

# Setup results directory
results_dir = project_root / "results" / "sample_size"
results_dir.mkdir(parents=True, exist_ok=True)
plots_dir = results_dir / "plots"
plots_dir.mkdir(exist_ok=True)
stats_dir = results_dir / "statistics"
stats_dir.mkdir(exist_ok=True)

print(f"Results will be saved to: {results_dir}")

# Import from Models folder
from Models.MC_Dropout import (
    MCDropoutRegressor,
    train_model,
    mc_dropout_predict,
    gaussian_nll,
    beta_nll,
    plot_toy_data,
    plot_uncertainties,
    normalize_x,
    normalize_x_data
)

from Models.Deep_Ensemble import (
    train_ensemble_deep,
    ensemble_predict_deep
)

from Models.BNN import (
    train_bnn,
    bnn_predict,
    normalize_x as bnn_normalize_x,
    normalize_x_data as bnn_normalize_x_data
)

from Models.BAMLSS import (
    fit_bamlss,
    bamlss_predict
)

from utils.device import get_device
from utils.plotting import plot_toy_data, plot_uncertainties_no_ood
import utils.results_save as results_save_module
from utils.results_save import save_plot, save_statistics, save_summary_text, save_summary_statistics

# Import helper functions for sample size experiments
from utils.sample_size_experiments import (
    run_mc_dropout_sample_size_experiment,
    run_deep_ensemble_sample_size_experiment,
    run_bnn_sample_size_experiment,
    run_bamlss_sample_size_experiment
)

# Set the module-level directories for results_save
results_save_module.plots_dir = plots_dir
results_save_module.stats_dir = stats_dir


## Device Setup


In [ ]:
device = get_device()


## Generate Toy Datasets


In [ ]:
# Reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

# ----- Data generation for linear function with homo/heteroscedastic noise -----
# f(x) = 0.7x + 0.5
# noise_type: 'homoscedastic' (σ(x) = 0.20) or 'heteroscedastic' (σ(x) = 0.10 + 0.2(0.5 + 0.5sin(x)))
def generate_toy_regression(n_train=1000, train_range=(0.0, 10.0), 
                           grid_points=1000, noise_type='heteroscedastic', type = "linear"):
    low, high = train_range
    x_train = np.random.uniform(low, high, size=(n_train, 1))
    
    if type == "linear":
        # Linear function: f(x) = 0.7x + 0.5
        f_clean = lambda x: 0.7 * x + 0.5
        y_clean_train = f_clean(x_train)
    elif type == "sin":
        f_clean = lambda x:  x * np.sin(x) + x
        y_clean_train = f_clean(x_train)
    else:
        raise ValueError("type must be 'linear', 'sin'")

    # Define noise variance σ²(x)
    if noise_type == 'homoscedastic':
        # Homoscedastic: σ(x) = 0.8
        sigma = 1
        sigma_train = np.full_like(x_train, sigma)
    elif noise_type == 'heteroscedastic':
        # Heteroscedastic: 
        sigma_train = np.abs(2.5 * np.sin(0.5*x_train +5))
    else:
        raise ValueError("noise_type must be 'homoscedastic' or 'heteroscedastic'")
    
    # Generate noise: ε | x ~ N(0, σ²(x))
    epsilon = np.random.normal(0.0, sigma_train, size=(n_train, 1))
    y_train = y_clean_train + epsilon

    # Dense evaluation grid within training range
    x_grid = np.linspace(train_range[0], train_range[1], grid_points).reshape(-1, 1)
    y_grid_clean = f_clean(x_grid)

    return (x_train.astype(np.float32), y_train.astype(np.float32),
            x_grid.astype(np.float32), y_grid_clean.astype(np.float32))


In [ ]:
# Polynomial function with homoscedastic noise
x_train_homo, y_train_homo, x_grid_homo, y_clean_homo = generate_toy_regression(
    n_train=1000, 
    train_range=(-5,10), 
    noise_type='homoscedastic',
    type = "sin"
)

# Polynomial function with heteroscedastic noise (default - used in most experiments)
x_train, y_train, x_grid, y_clean = generate_toy_regression(
    n_train = 1000, 
    train_range=(-5,10), 
    noise_type='heteroscedastic',
    type = "sin"
)

In [ ]:
#plot_toy_data(x_train_homo, y_train_homo, x_grid_homo, y_clean_homo, title="Toy Regression Data Homescedastic (n=1000)")

In [ ]:
#plot_toy_data(x_train, y_train, x_grid, y_clean, title="Toy Regression Data Heteroscedastic (n=1000)")

### Set parameters

In [ ]:
percentages=[5,10, 25, 50, 100]
percentages_bnn=[10, 100] # run for just two variations for BNNs due to computational cost
n_train_full = 1000
train_range = (-5,10)
grid_points = 1000
seed = 42
torch.manual_seed(seed)




## MC Dropout - Homoscedastic


In [ ]:
## MC Dropout - Homoscedastic

run_mc_dropout_sample_size_experiment(
    generate_toy_regression_func=generate_toy_regression,
    function_types=['linear', 'sin'],
    noise_type='homoscedastic',
    percentages=percentages,
    n_train_full = n_train_full,
    train_range=train_range,
    grid_points=grid_points,
    seed=seed,
    beta=0.5,
    lr=1e-3,
    batch_size=32
)

### hetero ###

run_mc_dropout_sample_size_experiment(
    generate_toy_regression_func=generate_toy_regression,
    function_types=['linear', 'sin'],
    noise_type='heteroscedastic',
    percentages=percentages,
    n_train_full = n_train_full,
    train_range=train_range,
    grid_points = grid_points,
    seed=seed,
    beta=0.5,
    lr=1e-3,
    batch_size=32
)


## Deep Ensemble 


In [ ]:
### homo ###

run_deep_ensemble_sample_size_experiment(
    generate_toy_regression_func=generate_toy_regression,
    function_types=['linear', 'sin'],
    noise_type='homoscedastic',
    percentages=percentages,
    n_train_full = n_train_full,
    train_range=train_range,
    grid_points=grid_points,
    seed=seed,
    beta=0.5,
    batch_size=32
)

### hetro ###

run_deep_ensemble_sample_size_experiment(
    generate_toy_regression_func=generate_toy_regression,
    function_types=['linear', 'sin'],
    noise_type='heteroscedastic',
    percentages=percentages,
    n_train_full = n_train_full,
    train_range=train_range,
    grid_points=grid_points,
    seed=seed,
    beta=0.5,
    batch_size=32,
)


## BAMLSS 


In [ ]:
### homoscedastic ###

run_bamlss_sample_size_experiment(
    generate_toy_regression_func=generate_toy_regression,
    function_types=['linear', 'sin'],
    noise_type='homoscedastic',
    percentages=percentages_bnn,
    n_train_full = n_train_full,
    train_range=train_range,
    grid_points=grid_points,
    seed=seed,
    n_iter=12000,
    burnin=200,
    thin=10,
    nsamples=500
)

### heteroscedastic ###

run_bamlss_sample_size_experiment(
    generate_toy_regression_func=generate_toy_regression,
    function_types=['linear', 'sin'],
    noise_type='heteroscedastic',
    percentages=percentages_bnn,
    n_train_full = n_train_full,
    train_range=train_range,
    grid_points=grid_points,
    seed=seed,
    n_iter=12000,
    burnin=2000,
    thin=10,
    nsamples=500
)



## BNN

In [ ]:
### Homoscedastic ###

run_bnn_sample_size_experiment(
    generate_toy_regression_func=generate_toy_regression,
    function_types=['linear', 'sin'],
    noise_type='homoscedastic',
    percentages=percentages_bnn,
    n_train_full = n_train_full,
    train_range=train_range,
    grid_points=grid_points,
    seed=seed,
    hidden_width=16,
    weight_scale=1.0,
    warmup=500,
    samples=500,
    chains=4
)

### Heteroscedastic ###

run_bnn_sample_size_experiment(
    generate_toy_regression_func=generate_toy_regression,
    function_types=['linear', 'sin'],
    noise_type='heteroscedastic',
    percentages=percentages_bnn,
    n_train_full = n_train_full,
    train_range=train_range,
    grid_points=grid_points,
    seed=seed,
    hidden_width=16,
    weight_scale=1.0,
    warmup=500,
    samples=500,
    chains=4
)
